In [58]:
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)

In [70]:
df_raw_2015 = pd.read_excel('华泰证券2015-2019.xlsx', dtype={'证券代码': np.object, '成交数量': np.float64})
# 强制保证基金代码的字符串类型
df_raw_2015['证券代码'] = df_raw_2015['证券代码'].apply(lambda x: str(x))
# ['流水号', '发生日期', '证券名称', '证券代码', '买卖标志', '业务名称', '成交价格', '成交数量', '发生金额',
#       '剩余金额', '佣金', '股东代码', '备注']
# print(df_raw_2015.info())
# print()

df_raw_current = pd.read_excel('华泰证券.xlsx', parse_dates=True, dtype={'发生日期': np.object, '证券代码': np.object, '成交数量': np.float64})
# 四费合一
df_raw_current['佣金'] = df_raw_current['佣金'] + df_raw_current['印花税'] + df_raw_current['过户费'] + df_raw_current['其他费']
# 强制保证基金代码的字符串类型
df_raw_current['证券代码'] = df_raw_current['证券代码'].apply(lambda x: str(x))
df_raw_current = df_raw_current[['流水号', '发生日期', '证券名称', '证券代码', '买卖标志', '业务名称', '成交价格', '成交数量', '发生金额',
       '剩余金额', '佣金', '股东代码', '备注']]
# 日期统一为 datetime64 类型
df_raw_current['发生日期'] = df_raw_current['发生日期'].apply(lambda x: pd.datetime(year=int(x[0:4]),month=int(x[4:6]),day=int(x[6:8])))
df_raw_current['发生日期'] = df_raw_current['发生日期'].astype(np.datetime64)
# print(df_raw_current.info())

# ['流水号', '发生日期', '证券名称', '证券代码', '买卖标志', '业务名称', '成交价格', '成交数量', '发生金额',
#      '剩余金额', '佣金', '股东代码',  '备注', '印花税', '过户费', '其他费', '币种']
df_raw = pd.concat([df_raw_2015, df_raw_current], ignore_index=True)

In [62]:
# 读取分类表
df_category = pd.read_excel('资产配置分类表.xlsx',dtype={'基金代码': np.object, '分类ID': np.int64})
# df_category.基金代码.unique()
# 基金名称	基金简称	基金代码	一级分类	二级分类	三级分类	分类ID	市场

In [88]:
# ['银行转存', '银行转取', '利息归本', '基金资金拨入', '基金资金拨出', '资管转让资金上账']
# df_sub[df_sub['业务名称'].isin(['利息归本', '基金资金拨入', '基金资金拨出', '资管转让资金上账'])]
# 基本上，证券名称为 " " 的记录，不是银行存取款，就是看也看不懂的。忽略吧
df_temp = df_raw[~(df_raw['证券名称'] == ' ')]
# 货币基金
df_money_fund = df_temp[df_temp['证券名称'].isin(['银华日利', '紫金货币', '天天发1', 'GC007', 'Ｒ-001', 'GC001', 
                                              'Ｒ-003', '现金添富', '华宝添益', '添富快线'])]
# 非货币基金
df_temp = df_temp[~df_temp.index.isin(df_money_fund.index)]

def occur_money_calc(x):
    money = round(np.abs(x['nav_unit'] * x['volume']), 2)
    if x['deal_type'] == '买入':
        return round(money + x['fee'], 2)
    elif x['deal_type'] == '卖出':
        return round(money - x['fee'], 2)
    elif x['deal_type'] == '分红':
        return round(np.abs(x['发生金额']) - x['fee'], 2)
    else:
        return 'NA'
    
def deal_money_calc(x):
    if x['deal_type'] == '分红':
        return round(np.abs(x['发生金额']), 2)
    else:
        money = round(np.abs(x['nav_unit'] * x['volume']), 2)
        return money

def deal_type_calc(x):
    if x in ['买入', '证券买入']:
        return '买入'
    elif x in ['卖出', '证券卖出', '开放基金赎回']:
        return '卖出'
    elif x in ['开放基金赎回返款', '股息入帐']:
        return '分红'
    pass

df_temp[df_temp['业务名称'].isin(['开放基金赎回返款', '开放基金赎回','股息入帐'])]

# 调整输出
# {'id': 1, 'date': '2020-02-03', 'time': '09:53:57', 'code': '162411', 
# 'name': '华宝油气', 'deal_type': '买入', 'nav_unit': 0.36, 'nav_acc': 0.36, 
# 'volume': 35500.0, 'deal_money': 12780.0, 'fee': 1.28, 'occur_money': 12781.28, 
# 'account': '华宝证券', 'category1': '商品', 'category2': '商品', 'category3': '原油', 
# 'category_id': 613, 'note': '无'}

df_temp['id'] = df_temp.reset_index().index + 1
df_temp['date'] = df_temp['发生日期']
df_temp['time'] = '9:30:00'
df_temp['code'] = df_temp['证券代码']
df_temp['name'] = df_temp['证券名称']
df_temp['deal_type'] = df_temp['业务名称'].apply(lambda x: deal_type_calc(x))
df_temp['nav_unit'] = df_temp['成交价格']
df_temp['nav_acc'] = df_temp['成交价格']
df_temp['volume'] = np.abs(df_temp['成交数量'])
df_temp['fee'] = df_temp['佣金']
df_temp['occur_money'] = df_temp.apply(occur_money_calc, axis=1)
df_temp['deal_money'] = df_temp.apply(deal_money_calc, axis=1)
df_temp['account'] = '华泰证券'
df_temp['note'] = '无'
# 补充一二三级分类
df_temp = pd.merge(df_temp, df_category, left_on='证券代码', right_on='基金代码', how='left')
df_temp = df_temp.rename(columns={'一级分类': 'category1', '二级分类': 'category2', '三级分类': 'category3', '分类ID': 'category_id'})
# 
df_temp = df_temp[['id','date','time','code','name','deal_type','nav_unit','nav_acc','volume','deal_money','fee','occur_money','account','category1','category2','category3','category_id','note']]
df_temp

# # 输出 csv 文件
# df_temp.to_csv('huatai_output.csv')
# # 输出 json 数据
# json_list = json.loads(df_temp.to_json(orient='records', force_ascii=False))
# [print(x) for x in json_list]


,id,date,time,code,name,deal_type,nav_unit,nav_acc,volume,deal_money,fee,occur_money,account,category1,category2,category3,category_id,note
0,1,2015-08-24,9:30:00,518880,黄金ETF,买入,2.411,2.411,4000.0,9644.00,2.03,9646.03,华泰证券,商品,商品,黄金,611,无
1,2,2015-08-31,9:30:00,510500,500ETF,买入,6.825,6.825,2000.0,13650.00,2.87,13652.87,华泰证券,A 股,中小盘股,中证500,121,无
2,3,2015-09-01,9:30:00,510500,500ETF,卖出,6.395,6.395,2000.0,12790.00,2.87,12787.13,华泰证券,A 股,中小盘股,中证500,121,无
3,4,2015-09-08,9:30:00,518880,黄金ETF,卖出,2.309,2.309,4000.0,9236.00,2.02,9233.98,华泰证券,商品,商品,黄金,611,无
4,5,2015-09-23,9:30:00,000836,鑫茂科技,买入,16.570,16.570,1200.0,19884.00,5.00,19889.00,华泰证券,A 股,股票,股票,151,无
5,6,2015-09-24,9:30:00,000836,鑫茂科技,卖出,18.060,18.060,1200.0,21672.00,31.02,21640.98,华泰证券,A 股,股票,股票,151,无
6,7,2015-10-08,9:30:00,510500,500ETF,买入,6.490,6.490,6000.0,38940.00,8.18,38948.18,华泰证券,A 股,中小盘股,中证500,121,无
7,8,2015-10-19,9:30:00,510500,500ETF,买入,7.171,7.171,3200.0,22947.20,4.82,22952.02,华泰证券,A 股,中小盘股,中证500,121,无
8,9,2015-10-21,9:30:00,600879,航天电子,买入,18.050,18.050,300.0,5415.00,5.11,5420.11,华泰证券,A 股,股票,股票,151,无
9,10,2015-10-21,9:30:00,510500,500ETF,买入,7.000,7.000,700.0,4900.00,1.03,4901.03,华泰证券,A 股,中小盘股,中证500,121,无
